# A Recommender System for Coffee Shop

In [1]:
!pip install folium

In [2]:
# importing libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geopy.geocoders # convert an address into latitude and longitude values
import geocoder
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries are imported.')

Libraries are imported.


### Postal Codes in Toronto

In [3]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

soup = BeautifulSoup(raw_wikipedia_page,'lxml')
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)
latitude = []
longitude = []
for elem in Postcode_u:
# initialize your variable to None
    lat_lng_coords = None

# loop until you get the coordinates
    while (lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(elem))
        lat_lng_coords = g.latlng
        # print(lat_lng_coords)

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u,
              'Latitude': latitude, 'Longitude':longitude}
df_toronto = pd.DataFrame.from_dict(toronto_dict)


df_toronto.head()

,Borough,Latitude,Longitude,Neighbourhood,Postcode
0,Scarborough,43.727929,-79.262029,"Ionview, Kennedy Park",M1K
1,North York,43.756303,-79.565963,Humber Summit,M9L
2,Downtown Toronto,43.640816,-79.381752,"Toronto Islands, Union Station",M5J
3,Downtown Toronto,43.651494,-79.375418,St. James Town,M5C
4,Etobicoke,43.636258,-79.498509,"Humber Bay, King's Mill Park, Mimico NE, Old M...",M8Y


### Create a Map of Toronto City (with its Postal Codes' Regions)

In [4]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)
#map_toronto
# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
map_toronto

# Selecting Scarorough in Toronto 

In [6]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough']
scarborough_data.head()

,Borough,Latitude,Longitude,Neighbourhood,Postcode
0,Scarborough,43.727929,-79.262029,"Ionview, Kennedy Park",M1K
6,Scarborough,43.784535,-79.160497,"Highland Creek, Rouge Hill, Port Union",M1C
7,Scarborough,43.692657,-79.264848,Birch Cliff,M1N
9,Scarborough,43.770992,-79.216917,Woburn,M1G
13,Scarborough,43.773136,-79.239476,Cedarbrae,M1H


### Create Map of Scarborough

In [7]:
address_scar = 'Scarborough, Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of "Scarborough" are: {}, {}.'.format(latitude_scar, longitude_scar))

map_Scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11.5)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Scarborough)  
    
map_Scarborough

The geograpical coordinate of "Scarborough" are: 43.773077, -79.257774.


In [14]:
# @hidden_cell
CLIENT_ID = 'K2WHCUUTTB50E2KHP4TZ3A0B3HNJTDR25LT4PJ5PVEIWLIBY'# your Foursquare ID
CLIENT_SECRET = 'H11ZJCQG01LBUTIYXP2ANLXMSY2ANPXAOIGN5IMJC02AROZM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [15]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;


### Venues in the Neighborhoods inside Scarborough

In [16]:
print('Crawling different neighborhoods inside "Scarborough"')
Scarborough_foursquare_dataset = foursquare_crawler(list(scarborough_data['Postcode']),
                                                   list(scarborough_data['Neighbourhood']),
                                                   list(scarborough_data['Latitude']),
                                                   list(scarborough_data['Longitude']),)


Crawling different neighborhoods inside "Scarborough"
1.
Data is Obtained, for the Postal Code M1K (and Neighborhoods Ionview, Kennedy Park) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M1C (and Neighborhoods Highland Creek, Rouge Hill, Port Union) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M1N (and Neighborhoods Birch Cliff) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M1G (and Neighborhoods Woburn) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M1H (and Neighborhoods Cedarbrae) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M1M (and Neighborhoods Cliffcrest, Cliffside) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M1W (and Neighborhoods Steeles West) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M1S (and Neighborhoods Agincourt) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M1R (and Neighborhoods Maryvale, Wexford) SUCCESSFULLY.
10.
Data is Obtained, for the Postal Code M1T (and Neighborhoods Tam O'Shanter) SU


## Saving results of Foursquare

In [17]:
import pickle
with open("Scarborough_foursquare_dataset.txt", "wb") as fp:   #Pickling
    pickle.dump(Scarborough_foursquare_dataset, fp)
print('Received Data from Internet is Saved to Computer.')  

Received Data from Internet is Saved to Computer.


In [18]:
with open("Scarborough_foursquare_dataset.txt", "rb") as fp:   # Unpickling
    Scarborough_foursquare_dataset = pickle.load(fp)

In [19]:

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])

    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']

            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)

    return(result_df)

In [20]:
scarborough_venues = get_venue_dataset(Scarborough_foursquare_dataset)

Number of Venuse in Coordination "M1K" Posal Code and "Ionview, Kennedy Park" Negihborhood(s) is:
25
Number of Venuse in Coordination "M1C" Posal Code and "Highland Creek, Rouge Hill, Port Union" Negihborhood(s) is:
4
Number of Venuse in Coordination "M1N" Posal Code and "Birch Cliff" Negihborhood(s) is:
15
Number of Venuse in Coordination "M1G" Posal Code and "Woburn" Negihborhood(s) is:
8
Number of Venuse in Coordination "M1H" Posal Code and "Cedarbrae" Negihborhood(s) is:
27
Number of Venuse in Coordination "M1M" Posal Code and "Cliffcrest, Cliffside" Negihborhood(s) is:
12
Number of Venuse in Coordination "M1W" Posal Code and "Steeles West" Negihborhood(s) is:
26
Number of Venuse in Coordination "M1S" Posal Code and "Agincourt" Negihborhood(s) is:
43
Number of Venuse in Coordination "M1R" Posal Code and "Maryvale, Wexford" Negihborhood(s) is:
29
Number of Venuse in Coordination "M1T" Posal Code and "Tam O'Shanter" Negihborhood(s) is:
33
Number of Venuse in Coordination "M1P" Posal 

### Showing Venues for Each Neighborhood in Scarborough

In [21]:
scarborough_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M1K,"Ionview, Kennedy Park",43.727929,-79.262029,Tim Hortons,This spot is popular,Coffee Shop,850
1,M1K,"Ionview, Kennedy Park",43.727929,-79.262029,Chung Moi,This spot is popular,Chinese Restaurant,764
2,M1K,"Ionview, Kennedy Park",43.727929,-79.262029,Dollarama,This spot is popular,Discount Store,784
3,M1K,"Ionview, Kennedy Park",43.727929,-79.262029,Giant Tiger,This spot is popular,Department Store,342
4,M1K,"Ionview, Kennedy Park",43.727929,-79.262029,Subway,This spot is popular,Sandwich Place,674


In [22]:
scarborough_venues.to_csv('scarborough_venues.csv')

In [23]:
scarborough_venues = pd.read_csv('scarborough_venues.csv')

In [24]:
neigh_list = list(scarborough_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside Scarborough:')
print(len(neigh_list))
print('List of Neighborhoods inside Scarborough:')
neigh_list

Number of Neighborhoods inside Scarborough:
16
List of Neighborhoods inside Scarborough:


['Ionview, Kennedy Park',
 'Highland Creek, Rouge Hill, Port Union',
 'Birch Cliff',
 'Woburn',
 'Cedarbrae',
 'Cliffcrest, Cliffside',
 'Steeles West',
 'Agincourt',
 'Maryvale, Wexford',
 "Tam O'Shanter",
 'Dorset Park, Scarborough Town Centre, Wexford Heights',
 'Clairlea, Golden Mile, Oakridge',
 'Agincourt North, Milliken',
 'Scarborough Village',
 'Rouge, Malvern',
 'Morningside, West Hill']

In [25]:
neigh_venue_summary = scarborough_venues.groupby('Neighborhood').count()
neigh_venue_summary.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
Agincourt,43,43,43,43,43,43,43
"Agincourt North, Milliken",28,28,28,28,28,28,28
Birch Cliff,15,15,15,15,15,15,15
Cedarbrae,27,27,27,27,27,27,27
"Clairlea, Golden Mile, Oakridge",27,27,27,27,27,27,27


In [26]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(scarborough_venues['Venue Category'].unique())

There are 117 uniques categories.
Here is the list of different categories:


['Coffee Shop',
 'Chinese Restaurant',
 'Discount Store',
 'Department Store',
 'Sandwich Place',
 'Burger Joint',
 'Bank',
 'Fast Food Restaurant',
 'Hobby Shop',
 'Pizza Place',
 'Train Station',
 'Grocery Store',
 'Hockey Arena',
 'Light Rail Station',
 'Metro Station',
 'Bus Line',
 'Rental Car Location',
 'Asian Restaurant',
 'Italian Restaurant',
 'Breakfast Spot',
 'Playground',
 'Café',
 'Thai Restaurant',
 'Park',
 'General Entertainment',
 'Gym',
 'Ice Cream Shop',
 'Restaurant',
 'Diner',
 'Skating Rink',
 'College Stadium',
 'Convenience Store',
 'Gym Pool',
 'Pharmacy',
 'Indian Restaurant',
 'Electronics Store',
 'Hakka Restaurant',
 'Caribbean Restaurant',
 'Athletics & Sports',
 'Music Store',
 'Bakery',
 'Fried Chicken Joint',
 'Wings Joint',
 'Yoga Studio',
 'German Restaurant',
 'Sporting Goods Shop',
 'Beach',
 'Furniture / Home Store',
 'Cajun / Creole Restaurant',
 'Sports Bar',
 'Hotpot Restaurant',
 'Japanese Restaurant',
 'Other Great Outdoors',
 'Auto Garage',

In [28]:
scarborough_onehot = pd.get_dummies(data = scarborough_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
scarborough_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,African Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,General Entertainment,German Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Music Store,Noodle House,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Rental Service,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Shop,Wings Joint,Yoga Studio
0,0,M1K,"Ionview, Kennedy Park",43.727929,-79.262029,Tim Hortons,This spot is popular,850,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M1K,"Ionview, Kennedy Park",43.727929,-79.262029,Chung Moi,This spot is popular,764,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M1K,"Ionview, Kennedy Park",43.727929,-79.262029,Dollarama,This spot is popular,784,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,M1K,"Ionview, Kennedy Park",43.727929,-79.262029,Giant Tiger,This spot is popular,342,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,M1K,"Ionview, Kennedy Park",43.727929,-79.262029,Subway,This spot is popular,674,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
# This list is created manually, but would be brought in from a csv normally
important_list_of_features = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'Tea Room',
 'Coffee Shop',
 'Event Space',
'Dessert Shop',
'Hobby Shop',
 'Park',
'Gym',
'Shop & Service',
 'Bakery',
 'Shopping Mall',
'Playground',
'Electronics Store',
'Gym / Fitness Center',
 'Pet Store',
'Bank',
 'Beer Store',
'Brewery',
 'Bowling Alley',
'Furniture / Home Store',
 'Wine Shop',
'Light Rail Station',
 'Clothing Store',
 'Plaza',
'Music Store',
'Yoga Studio',
 'Rental Car Location',
 'Bus Line',
'Train Station',
 'Metro Station',
 'Beach',
 'Ice Cream Shop',
'Sports Bar',
'Lounge',
'Pool',
'Pool Hall',
'Skating Rink',
'Bubble Tea Shop',
 'Spa',
'Fruit & Vegetable Store',
 'Arts & Crafts Store',
 'Café',
 'General Entertainment',
'College Stadium',
'Fish Market',
'Bar',
 'Smoke Shop',
'Bus Station',
'Hookah Bar',
'Golf Course',
'Food & Drink Shop',
 'Liquor Store',
'Auto Garage',
 'Tennis Court']

### Updating the One-hot Encoded DataFrame and
### Grouping the Data by Neighborhoods

In [30]:
scarborough_onehot = scarborough_onehot[important_list_of_features].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()


scarborough_onehot.head()

,Tea Room,Coffee Shop,Event Space,Dessert Shop,Hobby Shop,Park,Gym,Shop & Service,Bakery,Shopping Mall,Playground,Electronics Store,Gym / Fitness Center,Pet Store,Bank,Beer Store,Brewery,Bowling Alley,Furniture / Home Store,Wine Shop,Light Rail Station,Clothing Store,Plaza,Music Store,Yoga Studio,Rental Car Location,Bus Line,Train Station,Metro Station,Beach,Ice Cream Shop,Sports Bar,Lounge,Pool,Pool Hall,Skating Rink,Bubble Tea Shop,Spa,Fruit & Vegetable Store,Arts & Crafts Store,Café,General Entertainment,College Stadium,Fish Market,Bar,Smoke Shop,Bus Station,Hookah Bar,Golf Course,Food & Drink Shop,Liquor Store,Auto Garage,Tennis Court
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,0,1,0,0,0,1,0,0,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Agincourt North, Milliken",1,1,1,1,1,2,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Birch Cliff,0,0,0,0,0,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0
Cedarbrae,0,3,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Clairlea, Golden Mile, Oakridge",0,3,0,0,0,2,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0


In [31]:
feat_name_list = list(scarborough_onehot.columns)
coffee_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Coffee Shop') != (-1):
        coffee_list.append(value)
        
scarborough_onehot['Total Coffee Shop'] = scarborough_onehot[coffee_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = coffee_list)


feat_name_list = list(scarborough_onehot.columns)
joint_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
scarborough_onehot['Total Joints'] = scarborough_onehot[joint_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = joint_list)

### The Fully-Processed DataFrame

In [32]:
scarborough_onehot

,Tea Room,Event Space,Dessert Shop,Hobby Shop,Park,Gym,Shop & Service,Bakery,Shopping Mall,Playground,Electronics Store,Gym / Fitness Center,Pet Store,Bank,Beer Store,Brewery,Bowling Alley,Furniture / Home Store,Wine Shop,Light Rail Station,Clothing Store,Plaza,Music Store,Yoga Studio,Rental Car Location,Bus Line,Train Station,Metro Station,Beach,Ice Cream Shop,Sports Bar,Lounge,Pool,Pool Hall,Skating Rink,Bubble Tea Shop,Spa,Fruit & Vegetable Store,Arts & Crafts Store,Café,General Entertainment,College Stadium,Fish Market,Bar,Smoke Shop,Bus Station,Hookah Bar,Golf Course,Food & Drink Shop,Liquor Store,Auto Garage,Tennis Court,Total Coffee Shop,Total Joints
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,0,0,0,0,1,0,0,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0
"Agincourt North, Milliken",1,1,1,1,2,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0
Birch Cliff,0,0,0,0,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0.0
Cedarbrae,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0.0
"Clairlea, Golden Mile, Oakridge",0,0,0,0,2,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,3,0.0
"Cliffcrest, Cliffside",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
"Dorset Park, Scarborough Town Centre, Wexford Heights",0,0,0,0,1,0,0,1,0,0,3,1,1,1,1,1,1,4,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0.0
"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
"Ionview, Kennedy Park",0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0.0


In [33]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(scarborough_onehot)


In [34]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = scarborough_onehot.columns
means_df.index = ['Group 1','Group 2','Group 3','Group 4','Group 5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Tea Room,Event Space,Dessert Shop,Hobby Shop,Park,Gym,Shop & Service,Bakery,Shopping Mall,Playground,Electronics Store,Gym / Fitness Center,Pet Store,Bank,Beer Store,Brewery,Bowling Alley,Furniture / Home Store,Wine Shop,Light Rail Station,Clothing Store,Plaza,Music Store,Yoga Studio,Rental Car Location,Bus Line,Train Station,Metro Station,Beach,Ice Cream Shop,Sports Bar,Lounge,Pool,Pool Hall,Skating Rink,Bubble Tea Shop,Spa,Fruit & Vegetable Store,Arts & Crafts Store,Café,General Entertainment,College Stadium,Fish Market,Bar,Smoke Shop,Bus Station,Hookah Bar,Golf Course,Food & Drink Shop,Liquor Store,Auto Garage,Tennis Court,Total Coffee Shop,Total Joints,Total Sum
G4,0.000,0.000,0.000,0.0,1.00,0.00,0.000,1.0,0.000,0.000,3.000,1.0,1.0,1.0,1.000,1.0,1.0,4.0,1.0,1.000,1.0,1.0,0.000,0.000,0.00,0.0,0.000,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,3.000,0.0,22.00
G5,0.000,0.000,0.000,0.0,1.00,0.00,0.000,2.0,3.000,0.000,0.000,0.0,0.0,1.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.00,0.0,0.000,0.00,0.0,0.000,0.0,1.0,1.0,1.0,1.0,1.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,1.000,0.0,13.00
G2,0.125,0.125,0.125,0.5,0.75,0.25,0.125,1.0,0.375,0.125,0.125,0.0,0.0,0.5,0.125,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.125,0.125,0.25,0.5,0.125,0.25,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.125,0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.125,0.0,0.125,0.0,0.0,0.125,0.125,2.375,0.0,9.25
G1,0.000,0.000,0.000,0.0,0.00,1.00,0.000,0.0,0.000,0.000,0.000,0.0,0.0,1.0,1.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.00,0.0,0.000,0.00,0.0,0.000,1.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.0,1.000,0.0,0.000,1.0,1.0,0.000,0.000,2.000,0.0,9.00
G3,0.000,0.000,0.000,0.0,0.60,0.20,0.000,0.0,0.000,0.200,0.000,0.0,0.0,0.2,0.000,0.0,0.2,0.2,0.0,0.000,0.0,0.0,0.000,0.000,0.20,0.0,0.200,0.00,0.4,1.000,0.2,0.0,0.0,0.0,0.2,0.0,0.000,0.000,0.000,0.4,0.200,0.2,0.2,0.2,0.2,0.200,0.2,0.000,0.0,0.0,0.000,0.000,0.400,0.0,6.00


## Result:
### Best area is Group 4
### Second Best area is Group 5
### Third Best area is Group 2

In [35]:
neigh_summary = pd.DataFrame([scarborough_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,Agincourt,5
1,"Agincourt North, Milliken",2
2,Birch Cliff,3
3,Cedarbrae,2
4,"Clairlea, Golden Mile, Oakridge",2
5,"Cliffcrest, Cliffside",3
6,"Dorset Park, Scarborough Town Centre, Wexford ...",4
7,"Highland Creek, Rouge Hill, Port Union",3
8,"Ionview, Kennedy Park",2
9,"Maryvale, Wexford",3


# Deducing Results:
## Best Neighborhood Are...

In [65]:
map_loc = folium.Map(location = [scarborough_venues['Neighborhood Latitude'][0], scarborough_venues['Neighborhood Longitude'][0]], zoom_start = 10.7)

In [66]:
neigh_summary[neigh_summary['Group'] == 4]

,Neighborhood,Group
6,"Dorset Park, Scarborough Town Centre, Wexford ...",4


In [67]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 4]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Neighborhood': 'Dorset Park, Scarborough Town Centre, Wexford Heights',
 'Neighborhood Latitude': 43.7574096,
 'Neighborhood Longitude': -79.27330400000001,
 'Postal Code': 'M1P'}

In [68]:
folium.CircleMarker(
        [scarborough_venues['Neighborhood Latitude'][0], scarborough_venues['Neighborhood Longitude'][0]],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_loc)  

In [69]:
map_loc